In [ ]:
import sagemaker

print(sagemaker.__version__)
role = sagemaker.get_execution_role()

In [ ]:
bucket = "julsimon-huggingface-dogscats"
prefix = "train2500"
classes = "['dog','cat']"

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.23-1',
                                     role=role,
                                     instance_type='ml.m5.4xlarge',
                                     instance_count=1)

In [ ]:
%%time
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code='image-processing.py',
                      outputs=[
                          ProcessingOutput(
                              output_name='train_data',
                              source='/opt/ml/processing/train'),
                          ProcessingOutput(
                              output_name='valid_data',
                              source='/opt/ml/processing/valid'),
                          ProcessingOutput(
                              output_name='test_data',
                              source='/opt/ml/processing/test')
                      ],
                      arguments=[
                          '--bucket', bucket,
                          '--prefix', prefix,
                          '--classes', classes
                      ]
                     )

In [ ]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])
    